<a href="https://colab.research.google.com/github/jingtang10/odhs-genai/blob/main/%5BODHS_D2_T1_05A_CL2_1%5D_Gemini_WHO_ANC_Guidelines_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the Gemini API SDK

The Python SDK for the Gemini API is contained in the [google-generativeai package](https://pypi.org/project/google-generativeai/). Install the dependency using pip.



In [ ]:
! pip install -U -q google-generativeai

## Import the libraries


In [ ]:
import os
import json
import pathlib
import textwrap
import pandas as pd
import time

import google.generativeai as genai
from google.generativeai import caching

from IPython.display import display
from IPython.display import Markdown
from typing import Union, List

from google.generativeai.types import RequestOptions
from google.api_core import retry, exceptions


## Set up your API key


To use the Gemini API, you'll need an API key. Store your API key in Colab Secrets named `GOOGLE_API_KEY`.   
If you don't have an API key or need help creating a Colab Secrets, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) guide.

In [ ]:
# passing the API key
try:
  from google.colab import userdata
  GOOGLE_API_KEY = userdata.get ('GOOGLE_API_KEY')
  genai.configure(api_key=GOOGLE_API_KEY)
except ImportError:
  pass

## Set environment variables
Set environment variables for
add huggingface


## Upload WHO's antenatal care guidelines file

Use the [`upload_file`](https://ai.google.dev/gemini-api/docs/document-processing?lang=python#upload-document) API to temporarily store WHO's antenatal care guidelines pdf file. This process produces a file reference that can be used to prompt a model.

In [ ]:
file_path = "9789241549912-eng.pdf" #@param {type:"string"}
display_name = "WHO recommendations on antenatal care for a positive pregnancy experience"

pdf_file = genai.upload_file(
    path=file_path,
    display_name=display_name,
)

file_ref = genai.get_file(name=pdf_file.name)

## Select a suitable gemini model

The Gemini API offers different models that are optimized for specific use cases. Here's a [brief overview of Gemini variants](https://ai.google.dev/gemini-api/docs/models/gemini?_gl=1*cyblbc*_up*MQ..&gclid=Cj0KCQjwsJO4BhDoARIsADDv4vB5i1gAcxplfDp37YCnHdYV1vFF_11JvdxwPjqBjujKpgMKrmDHM9caAlGLEALw_wcB) that are available.
To ensure your prompts work correctly, check the input and output token limits. Make sure your document and desired output fit within these limits.

We will be using [Gemini 1.5 pro](https://ai.google.dev/gemini-api/docs/models/gemini?_gl=1*cyblbc*_up*MQ..&gclid=Cj0KCQjwsJO4BhDoARIsADDv4vB5i1gAcxplfDp37YCnHdYV1vFF_11JvdxwPjqBjujKpgMKrmDHM9caAlGLEALw_wcB#gemini-1.5-pro) model in this tutorial to generate the questionnaires.

In [ ]:
model_name = "gemini-1.5-pro-latest" #@param ["gemini-1.5-pro-latest", "gemini-1.5-pro", "gemini-1.5-flash-latest", "gemini-1.5-flash"]
model_info = genai.get_model(f'models/{model_name}')

print(f"Model: {model_name}")
print(f"Input Token Limit: {model_info.input_token_limit}")
print(f"Output Token Limit: {model_info.output_token_limit}")

Model: gemini-1.5-pro-latest
Input Token Limit: 2000000
Output Token Limit: 8192


## Let's define some utility functions to perform repetitative tasks:

1.   Define the model
2.   Call the model and generate text



In [ ]:
def model_def(model_name, safety_settings, tools=None):
  model = genai.GenerativeModel(
    model_name=model_name,
    safety_settings=safety_settings,
    tools=tools
  )
  return model

In [ ]:
def format_items_for_prompt(items: list, item_type: str,model_name: str) -> str:
  """
  Formats a list of items (questions, etc.) into a string for the prompt.
  Handles different item types (e.g., "mcq", "sqa").
  """
  formatted_text = ""
  for item in items:
    formatted_text += f"Question: {item['Question']}\n"
    formatted_text += f"{item.get('Intervention', '')}\n"
    if item_type == "mcq":
      if model_name == 'gemini':
        formatted_text += "".join(
            [f"{option}: {item[option]}\n" for option in ["A", "B", "C", "D", "E"]]
        )
        formatted_text += f"Choice: {item.get('Choice', '')}\n\n" # Include Choice if present
      elif model_name == "gemma":
          formatted_text += "".join(
            [f"{option}: {item[option]}\n" for option in ["A", "B", "C", "D", "E"]]
        )
          formatted_text += f"Answer: \n"

    elif item_type == "sqa_eval":
      if model_name == 'gemini':
        formatted_text += f"Answer: {item.get('Answer', '')}\n\n"
      elif model_name == "gemma":
        formatted_text += f"Answer: {item.get('Answer_2b', '')}\n\n"

    elif item_type == "sqa":
      if model_name == 'gemini':
        formatted_text += f"Answer: {item.get('Answer', '')}\n\n"
      elif model_name == 'gemma':
        formatted_text += f"Answer: \n\n"

  return formatted_text

In [ ]:
def call_model_and_extract(
    chat_model: genai.GenerativeModel,
    prompt: list,
    function_name: str,
    generation_config: dict = None,
    tool_config: dict = None,
) -> list:
  """Calls the language model and extracts the results from the function call."""

  try:
    response = generate_text(
        chat_model,
        prompt,
        generation_config=generation_config,
        tool_config=tool_config,
    )

    if response.candidates[0].content.parts[0].function_call:
      function_call = response.candidates[0].content.parts[0].function_call
      extracted_results = type(function_call).to_dict(function_call)["args"][
          function_name
      ]
      return extracted_results
    else:
      return []  # Return empty list if no function call


  except Exception as e:
    print(f"Error calling model: {e}")
    return []  # Return empty list on error

## Generate text

* The Gemini API's client library offers built-in retry mechanisms for handling transient errors.

* The `generate_text` function sends a message to the chat model with the given prompt and
  configurations, and returns the response. It includes retry logic to handle
  transient errors.

* For more info on error handling, take a look at the [error_handling quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Error_handling.ipynb).

In [ ]:
@retry.Retry(
    predicate=retry.if_transient_error,
    initial=5,
    maximum=10,
    multiplier=2.0,
    timeout=100,
)

def generate_text(
    chat: genai.GenerativeModel,
    prompt: Union[List[str], str],
    generation_config: dict = None,
    tool_config: dict = None
):
  """Generates text using a chat model, with retry mechanism for transient errors.

  This function sends a prompt to a chat model and returns the generated response.
  It uses a retry decorator to handle transient errors, such as network issues,
  allowing the function to automatically retry the operation multiple times
  before giving up.

  Args:
    chat: The chat model object (an instance of `genai.Model`).
    prompt: The text prompt to send to the chat model. Can be a string or a list of strings.
    generation_config: (Optional) A dictionary containing configuration
        parameters for the text generation process. This might include settings
        like temperature, max tokens, etc. The specific format depends on the
        `chat` object (genai.Model).  See GenAI's documentation for details.
    tool_config: (Optional)  A dictionary containing configuration
        parameters for any tools that the chat model might use. The specific
        format depends on the `chat` object (genai.Model) and whether it
        supports tools.

  Returns:
    The response from the chat model (genai.Response).


  Raises:
    retry.RetryError: If the function fails to generate text after multiple
        retries due to persistent transient errors. The original exception
        that triggered the retries will be chained to the `RetryError`.
    Any other exception raised by `chat.generate_text`: If the `generate_text`
        method raises an exception that is not considered a transient error,
        the exception will be propagated directly without retries.
  """

  response = chat.send_message(
      prompt,
      generation_config=generation_config,
      tool_config=tool_config,
  )
  return response

## Customize Safety Settings

The Gemini API provides safety settings that you can adjust during the prototyping stage to determine if your application requires more or less restrictive safety configuration. You can adjust these settings across four filter categories to restrict or allow certain types of content.

To make this customization you must define a safety_settings and pass it to model initialization as below.

**Important:** To guarantee the Google commitment with the Responsible AI development and its [AI Principles](https://ai.google/responsibility/principles/), for some prompts Gemini will avoid generating the results even if you set all the filters to none.

In [ ]:

HARM_CATEGORY_DANGEROUS = "BLOCK_NONE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"]
HARM_CATEGORY_HARASSMENT = "BLOCK_NONE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"]
HARM_CATEGORY_HATE_SPEECH = "BLOCK_NONE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"]
HARM_CATEGORY_SEXUALLY_EXPLICIT = "BLOCK_NONE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"]
HARM_CATEGORY_DANGEROUS_CONTENT = "BLOCK_NONE" # @param ["BLOCK_NONE", "BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"]

safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": HARM_CATEGORY_DANGEROUS,
        },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": HARM_CATEGORY_HARASSMENT,
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": HARM_CATEGORY_HATE_SPEECH,
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": HARM_CATEGORY_SEXUALLY_EXPLICIT,
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": HARM_CATEGORY_DANGEROUS_CONTENT,
    },
]

## Configure text generation


Every prompt you send to the model includes [parameters]((https://ai.google.dev/gemini-api/docs/models/generative-models#model-parameters)) that control how the model generates responses. You can use [genai.GenerationConfig](https://ai.google.dev/api/generate-content#generationconfig) to configure these parameters. If you don't configure the parameters, the model uses default options, which can vary by model.

In [ ]:
temperature = 1 #@param {type:"slider", min:0, max:1, step:0.1}
top_p = 0.95 #@param {type:"slider", min:0, max:1, step:0.05}
top_k = 64 #@param {type:"integer"}
max_output_tokens = 8192 #@param {type:"integer"}

generation_config ={
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
}

## Evaluate the generated MCQ questions with Gemini 1.5 model

In this section we try to evaluate the questions generated from Gemini model and what context has been to used to frame the Multiple Choice Questions.

In [ ]:
def eval_questions(
    chat_model: genai.GenerativeModel,
    file_ref: genai.types.file_types.File,
    base_prompt: str,
    questions_dict_list: list,
    question_type: str,
    eval_type_function_name: str,
    eval_per_prompt: int,
    generation_config: dict = None,
    tool_config: dict = None,
    model_name: str = "gemini"
):
  evaluated_questions = []
  current_prompt = [file_ref, base_prompt]

  for idx in range(0, len(questions_dict_list), eval_per_prompt):
    formatted_text = format_items_for_prompt(
        questions_dict_list[idx:idx+eval_per_prompt],
        question_type,
        model_name
    )
    current_prompt.append(formatted_text)

    per_prompt_evaluated_questions = call_model_and_extract(
        chat_model,
        current_prompt,
        eval_type_function_name,
        generation_config=generation_config,
        tool_config=tool_config,
    )
    print(f"Evaluated {len(per_prompt_evaluated_questions)} questions in prompt {idx//eval_per_prompt+1}")

    evaluated_questions.extend(per_prompt_evaluated_questions)
    current_prompt = []

  return evaluated_questions


### Base Prompt

In [ ]:
eval_mcq_base_prompt = textwrap.dedent("""\
Task: Evaluate the provided multiple-choice questions using above extracted text corpus based on the following criteria:

Accuracy: Ensure the correct response aligns with the information presented in the provided text corpus.
Relevance: Verify that the question and answer choices are directly related to the stated intervention topic.
Clarity: Check if the question and answer choices are clear, concise, and avoid ambiguity.
Consistency: Ensure that the correct response is consistent with other relevant information in the provided text corpus.

Evaluation Format:

Question: [Question text]
Intervention Topic: [Intervention topic]
Correct Answer: [Correct option]
Context: [Page number or section reference where the correct answer can be found in the provided text corpus]

Example:

Question: According to the WHO recommendations, what is the recommended daily intake of calcium for pregnant women?
Intervention Topic: Nutritional interventions
Correct Answer: B
Context: Page 72, Section 3.1 of the WHO Guidelines for the Prevention and Management of Gestational Diabetes Mellitus
Note: To ensure accurate and relevant evaluations, please provide the specific text corpus that the questions are based on.
This will allow for a more precise assessment of the accuracy, relevance, clarity, and consistency of the questions and answers.
"""
)
print(eval_mcq_base_prompt)

Task: Evaluate the provided multiple-choice questions using above extracted text corpus based on the following criteria:

Accuracy: Ensure the correct response aligns with the information presented in the provided text corpus.
Relevance: Verify that the question and answer choices are directly related to the stated intervention topic.
Clarity: Check if the question and answer choices are clear, concise, and avoid ambiguity.
Consistency: Ensure that the correct response is consistent with other relevant information in the provided text corpus.

Evaluation Format:

Question: [Question text]
Intervention Topic: [Intervention topic]
Correct Answer: [Correct option]
Context: [Page number or section reference where the correct answer can be found in the provided text corpus]

Example:

Question: According to the WHO recommendations, what is the recommended daily intake of calcium for pregnant women?
Intervention Topic: Nutritional interventions
Correct Answer: B
Context: Page 72, Section 3.1

### Function Calling

#### Single Eval Schema

This schema defines the structure for evaluating multiple choice questions. It includes the following fields:

* Intervention: The topic or subject matter related to the question.
* Question: The actual multiple-choice question.
* Choice: The chosen option or answer for the question.
* Correct: A boolean value indicating whether the chosen answer is correct (1) or incorrect (0).
* Context: A reference or source where the correct answer can be found, such as a page number or section in a document.

In [ ]:
mcq_eval = genai.protos.Schema(
    type = genai.protos.Type.OBJECT,
    properties = {
        'Intervention':  genai.protos.Schema(type=genai.protos.Type.STRING),
        'Question':  genai.protos.Schema(type=genai.protos.Type.STRING),
        'Choice': genai.protos.Schema(type=genai.protos.Type.STRING),
        'Correct': genai.protos.Schema(type=genai.protos.Type.NUMBER),
        'Context': genai.protos.Schema(type=genai.protos.Type.STRING),
    },
    required=['Intervention', 'Question', 'Choice', 'Correct', 'Context']
)

#### Array Schema

In [ ]:
mcq_eval_schema = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=mcq_eval
)

In [ ]:
mcq_eval_database = genai.protos.FunctionDeclaration(
    name="mcq_eval_database",
    description=textwrap.dedent("""\
        Adds interventions, questions, answers, correctness and its context to the database.
        """),
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties = {
            'mcq_eval': mcq_eval_schema,
        }
    )
)

### Define the model with function declaration

In [ ]:
model = model_def(model_name, safety_settings, tools=[mcq_eval_database])
chat = model.start_chat(history=[])

### Load the csv file containing previously generated mcq questions

In [ ]:
mcq_questions_df = pd.read_csv("mcq_questions.csv")
mcq_questions_dict_list = mcq_questions_df.to_dict(orient="records")

### Evaluate in chat session

In [ ]:
eval_num_mcq_per_prompt = 20

mcq_evaluated_questions = eval_questions(
    chat_model=chat,
    file_ref=file_ref,
    base_prompt=eval_mcq_base_prompt,
    questions_dict_list=mcq_questions_dict_list,
    question_type="mcq",
    eval_type_function_name="mcq_eval",
    eval_per_prompt=eval_num_mcq_per_prompt,
    generation_config=generation_config,
    tool_config={"function_calling_config": {"mode": "ANY"}},
    model_name = 'gemini'
)

Evaluated 20 questions in prompt 1


Evaluated 20 questions in prompt 2


Evaluated 20 questions in prompt 3


Evaluated 20 questions in prompt 4


Evaluated 20 questions in prompt 5


In [ ]:
mcq_eval_df = pd.DataFrame(mcq_evaluated_questions)
mcq_eval_df

,Question,Choice,Correct,Context,Intervention
0,Which supplement is routinely recommended for ...,A,1.0,"Page 14, Section A. Nutritional interventions",Nutritional interventions
1,What does a healthy diet during pregnancy cons...,D,1.0,"Page 14, Section A. Nutritional interventions",Nutritional interventions
2,"For undernourished populations, which type of ...",B,1.0,"Page 20, Section A.1.3",Nutritional interventions
3,When is vitamin A supplementation recommended ...,B,1.0,"Page 29, Section A.4",Nutritional interventions
4,What is the recommended method for diagnosing ...,C,1.0,"Page 41, Section B.1.1",Maternal and fetal assessment
...,...,...,...,...,...
95,Which of the following may be recommended for ...,D,1.0,"Page 74, Recommendation D.1",Interventional measures for common physiologic...
96,Which statement about midwife-led continuity o...,A,1.0,"Page 89, Section E.2",Health systems interventions
97,What is a key consideration regarding task-shi...,D,1.0,"Page 99, Recommendation E.5",Health systems interventions
98,What is an accurate statement about group ante...,D,1.0,"Page 91, Recommendation E.3",Health systems interventions


In [ ]:
# Save the evaluations to a CSV file
mcq_eval_df.to_csv("mcq_eval.csv", index=False)

Evaluation results show that all of the MCQ questions were answered correctly by Gemini 1.5 pro model





## Evaluate the Short Question Answers generated


In this section we try to evaluate the questions generated from Gemini model and what context has been to used to frame the Short Question Answers.

### Base Prompt

In [ ]:
eval_sqa_base_prompt = textwrap.dedent("""\
Task: Evaluate the provided short answer responses using above extracted text corpus based on the following criteria:

Accuracy: Ensure the response aligns with the information presented in the provided text corpus.
Relevance: Verify that the response is directly related to the stated intervention topic.
Clarity: Check if the response is clear, concise, and avoids ambiguity.
Consistency: Ensure the response is consistent with other relevant information in the provided text corpus.

Evaluation Format:

Question: [Question text]
Intervention Topic: [Intervention topic]
Response: [Short answer response]
Grade: [1-5] (1: Poor, 2: Needs Improvement, 3: Satisfactory, 4: Good, 5: Excellent)
Context: [Page number or section reference where the response is supported or contradicted in the provided text corpus]

Example:

Question: According to the WHO recommendations, what is the recommended daily intake of calcium for pregnant women?
Intervention Topic: Nutritional interventions
Response: "Pregnant women should consume 1000 mg of calcium per day."
Grade: 4
Context: Page 75, Section 3.2 of the WHO Guidelines for the Prevention and Management of Gestational Diabetes Mellitus

Note: To ensure accurate and relevant evaluations, please provide the specific text corpus that the short answer responses are based on.
This will allow for a more precise assessment of the accuracy, relevance, clarity, and consistency of the responses.
"""
)
print(eval_sqa_base_prompt)

Task: Evaluate the provided short answer responses using above extracted text corpus based on the following criteria:

Accuracy: Ensure the response aligns with the information presented in the provided text corpus.
Relevance: Verify that the response is directly related to the stated intervention topic.
Clarity: Check if the response is clear, concise, and avoids ambiguity.
Consistency: Ensure the response is consistent with other relevant information in the provided text corpus.

Evaluation Format:

Question: [Question text]
Intervention Topic: [Intervention topic]
Response: [Short answer response]
Grade: [1-5] (1: Poor, 2: Needs Improvement, 3: Satisfactory, 4: Good, 5: Excellent)
Context: [Page number or section reference where the response is supported or contradicted in the provided text corpus]

Example:

Question: According to the WHO recommendations, what is the recommended daily intake of calcium for pregnant women?
Intervention Topic: Nutritional interventions
Response: "Pre

### Function Calling

#### SQA Eval Schema

In [ ]:
sqa_eval_single_schema = genai.protos.Schema(
    type = genai.protos.Type.OBJECT,
    properties = {
        'Intervention':  genai.protos.Schema(type=genai.protos.Type.STRING),
        'Question':  genai.protos.Schema(type=genai.protos.Type.STRING),
        'Answer': genai.protos.Schema(type=genai.protos.Type.STRING),
        'Grade': genai.protos.Schema(type=genai.protos.Type.NUMBER),
        'Context': genai.protos.Schema(type=genai.protos.Type.STRING),
    },
    required=['Intervention', 'Question', 'Answer', 'Grade', 'Context']
)

#### Array Schema

In [ ]:
sqa_eval_schema = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=sqa_eval_single_schema
)

#### Function Declaration

In [ ]:
sqa_eval_database = genai.protos.FunctionDeclaration(
    name="sqa_eval_database",
    description=textwrap.dedent("""\
        Adds interventions, questions, answers, grading and its context to the database.
        """),
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties = {
            'sqa_eval': sqa_eval_schema,
        }
    )
)

In [ ]:
sqa_questions_df = pd.read_csv("sqa_questions.csv")
sqa_questions_dict_list = sqa_questions_df.to_dict(orient="records")

### Define the model with function declaration

In [ ]:
model = model_def(model_name, safety_settings, tools=[sqa_eval_database])
chat = model.start_chat(history=[])

### Evaluate in chat session

In [ ]:
num_sqa_eval_per_prompt  = 20
sqa_evaluated_questions = eval_questions(
    chat_model=chat,
    file_ref=file_ref,
    base_prompt=eval_sqa_base_prompt,
    questions_dict_list=sqa_questions_dict_list,
    question_type="sqa_eval",
    eval_type_function_name="sqa_eval",
    eval_per_prompt=num_sqa_eval_per_prompt,
    generation_config=generation_config,
    tool_config={"function_calling_config": {"mode": "ANY"}},
    model_name = 'gemini'
)

Evaluated 20 questions in prompt 1


Evaluated 20 questions in prompt 2


Evaluated 20 questions in prompt 3


Evaluated 20 questions in prompt 4


Evaluated 20 questions in prompt 5


In [ ]:
sqa_eval_df = pd.DataFrame(sqa_evaluated_questions)
sqa_eval_df

,Question,Answer,Grade,Context,Intervention
0,What constitutes a healthy diet during pregnancy?,Pregnant women should consume a variety of foo...,5.0,Page 15,Nutritional interventions
1,What is the goal of nutritional counseling dur...,Nutritional counseling during pregnancy aims t...,5.0,Page 15,Nutritional interventions
2,Why is addressing undernutrition during pregna...,Undernutrition during pregnancy can lead to ad...,5.0,Page 18,Nutritional interventions
3,When is balanced energy and protein supplement...,Balanced energy and protein supplementation is...,5.0,Page 20,Nutritional interventions
4,What are the recommended methods for diagnosin...,Full blood count testing is the recommended me...,5.0,Page 41,Maternal and fetal assessment
...,...,...,...,...,...
95,What non-pharmacological approaches can help m...,Non-pharmacological interventions like compres...,5.0,Page 83,Interventions from common physiological symptoms
96,Why are woman-held case notes encouraged durin...,Woman-held case notes are recommended for preg...,5.0,Page 87,Health systems interventions to improve the ut...
97,Where are midwife-led continuity of care model...,Midwife-led continuity of care models are reco...,5.0,Page 89,Health systems interventions to improve the ut...
98,What incentives can be used to address healthc...,Policymakers should consider offering various ...,5.0,Page 100,Health systems interventions to improve the ut...


In [ ]:
sqa_eval_df.to_csv("sqa_eval.csv", index=False)